# Convolutional Neural Networks

If we have 32x32x3 colour image files, first layer of a NN would need of the order of 3000 parameters

Fix this problem by convolving the picture with a kernel. This assumes that picels closer together will generally have shared features, just as colour gradients.

The values of the kernel are learned, just like the weights, in a 'standard' neural network.

It is worth noting that, although using convolutional layers decreases that dimension of the parameter space, it introduces more hyperparameters (such as depth, number of different kernels, height and width of each kernel, kernel step size).

## Stride length

The stide length is the size of each step the kernel makes once a convolution of the specified region has been performed. For instance, if a stride length of 1 is used, then for any filter with dimension greater than one, there will be some overlap.

## Padding

When passing a kernel filter over the image, for certain stride lengths, the filter will not properly complete a pass over the whole image. To avoid this problem, one can use zero-padding to pad out the rest of the image with zeros to make sure the dimensions are correct for a given kernel size.

## Kernel depth

When performing a pass over an image, the filter passes over the height and width of the image, with specified dimensions respectively. However, it always passes over the entirity the input depth each time. Yet, a hyperparameter us the convolutional, or kernel, depth, which specifies the number of times the filter passes over the image and collects different information. Here, for each kernel layer there is an additional node, which might, for instance, learn to recognise faces, or wheels on a car, etc.. As an example, if we have images of the size 32x32x3 as input, and pass a 4x4 filter over it once (i.e. kernel depth of 1) with stride 2, then, since (32-4)/2 + 1 = 15 (where the +1 comes from the bias parameter), there will be 15x15 = 225 neurons in the layer. Moreover, each of these neurons will have 4x4x3 = 48 weights (plus a bias parameter). However, if we now have a kernel depth of 20 then there will be 15x15x20 = 4500 neurons in the layer (and a total of 4500x3 = 13500 weights to learn). Increasing the number of passes by the filter over the input can clearly dramatically increase the computational time of the training process, however, one, in theory, can decern more nuanced image features (but with overfitting being a potential danger). 

## Pooling

After a stated number of convolutional layers, we will implement a pooling layer. In particular, Max Pooling will be used, whereby the maximum value in a given region (often 2x2) is calculated and entered into a smaller matrix. Max Pooling further reduces the dimensionality of the data and can often smooth out small disjointed sections of the data. In downscaling the input in this way, can dramatically increase the amount to which the network becomes invariant to translations and rotations.

## Dropout

Dropout method will be used to regulate overfitting. Given a dropout probability, in a given Dropout iteration (which will be trated as a layer) the network will go through each parameter and ignore a node with a probabaility equal to the dropout probabaility. This will have particular effect in the dense layers. This method helps reduce overfitting in the model by reducing the learning of noise in the data.

For more information:

http://cs231n.github.io/convolutional-networks/

## Keras

Keras a modular and user firendly way of building neural networks quickly. It uses a TensorFlow backend (if you choose it to) which allows you the flexibility of TF with the useability of Keras. Keras allows the definition of additional information (for instance custom layer building) using TF.

## The CIFAR10 dataset

In [ ]:
from keras.utils import np_utils
import numpy as np

Load and split the data

In [ ]:
from keras.datasets import cifar10

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
num_train, height, width, depth = X_train.shape # there are 50000 training examples in CIFAR-10 
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = np.unique(y_train).shape[0] # there are 10 image classes

In [ ]:
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')

In [ ]:
X_train /= np.max(X_train) # Normalise data to [0, 1] range
X_test /= np.max(X_test) # Normalise data to [0, 1] range

In [ ]:
Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

In [ ]:
X_train.shape

Define hyperparameters

In [ ]:
batch_size = 32 # mini-batch learning
num_epochs = 3
kernel_size = 3
pool_size = 2 
conv_depth_1 = 32 # kernel depth of 32 in first set of conv layers
conv_depth_2 = 64 # kernel depth of 64 in second set of conv layers
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

Build the network

In [ ]:
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout
from keras.layers.advanced_activations import PReLU
import keras.layers

In [ ]:
inpt = Input(shape=(height,width,depth))

conv_1 = Convolution2D(conv_depth_1,(kernel_size, kernel_size),padding='same',activation='relu')(inpt)
conv_2 = Convolution2D(conv_depth_2,(kernel_size, kernel_size),padding='same',activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size = (pool_size,pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)

conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)

flat = keras.layers.Flatten()(drop_2)

hidden = Dense(hidden_size, activation = 'relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inputs=inpt, outputs=out)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(X_train, Y_train,                # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation
model.evaluate(X_test, Y_test, verbose=1)  # Evaluate the trained model on the test set!

For some more complex network examples see: http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html#43494641522d3130

## Fractional Max Pooling (FMP)

FMP is the method of varying the dimension of the pooling region as you pool across the input. FMP has been found to produce siginicantly improved results, at least when it comes to toy datastes like MNIST and CIFAR10 and is based off the hyperparameter of the ratio of the input parameter to the output paramenter. The size of the pooling region is generated either 'randomly' or 'pseudo-randomly'.

In the example were we take out pooling size to be 2x2, we are essentially downscaling to a size 25% that of the input. FMP allows for slower downscaling, as the effective pooling size (the average of the pooling region over the whole input) can take on non-integer value. This can dramatically reduce the risk of overfitting, at the expense of computational time.

In [ ]:
import tensorflow as tf

In [ ]:
def FMP(x,pooling_ratio):
    return tf.nn.fractional_max_pool(x,pooling_ratio)[0] #normally has three outputs of which the output is the first

In [ ]:
pooling_ratio = [1.0,1.44,1.44,1.0] 
#first and last entries must be 1.0 as tf doesn't pool over batch and channel dimensions

In [ ]:
from keras.models import Sequential

In [ ]:
model = Sequential()

model.add(Convolution2D(conv_depth_1, (kernel_size, kernel_size), strides=(1, 1), padding='same', activation='relu', input_shape = (32,32,3)))

#TODO: BUILD MODEL FOR FMP

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x=X_train, y=Y_train, batch_size=batch_size, epochs=num_epochs, verbose=1, validation_split=0.1)

model.evaluate(x=X_test, y=Y_test, verbose=1)

https://arxiv.org/pdf/1412.6071.pdf

https://www.tensorflow.org/api_docs/python/tf/nn/fractional_max_pool

https://stackoverflow.com/questions/44991470/using-tensorflow-layers-in-keras